In [1]:
# Resolución robusta de ROOT para importar utilidades del proyecto (si se usan)
from pathlib import Path
import sys

def find_root(start=None):
    p = Path(start or Path.cwd()).resolve()
    for _ in range(6):
        if (p / 'requirements.txt').exists() and (p / 'src').exists():
            ROOT = p
            break
        if (p / '.git').exists() and (p / 'src').exists():
            ROOT = p
            break
        p = p.parent
    else:
        ROOT = Path.cwd()

if 'ROOT' not in globals():
    ROOT = find_root()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

# 06 · Proyecciones Financieras (3 años)

Objetivo: comparar escenarios A (sin suscripción) vs B (con suscripción) con supuestos y sensibilidad.

In [ ]:
import pandas as pd
import plotly.express as px

# Supuestos (editables):
base_revenue_year0 = 140000  # aproximado sin suscripción
decline_rate = 0.08  # 8% anual por streaming
subs_years = [350, 490, 590]  # adopción B año 1-3
price = 180
variable_cost_per_customer = 80

# Escenario A (sin suscripción):
rev_A = [base_revenue_year0 * ((1 - decline_rate) ** i) for i in range(1,4)]

# Escenario B (con suscripción):
rev_B_subs = [n * price for n in subs_years]
var_costs_B = [n * variable_cost_per_customer for n in subs_years]
contrib_B = [r - c for r, c in zip(rev_B_subs, var_costs_B)]
# Agregar status-quo deteriorándose (opcional):
rev_B_total = [a + b for a, b in zip(rev_A, rev_B_subs)]

# Tabla resumen
df = pd.DataFrame({
    'Año': [1,2,3],
    'Escenario A (sin sub)': rev_A,
    'Escenario B (con sub)': rev_B_total,
    'Revenue suscripción': rev_B_subs,
    'Margen contribución sub': contrib_B,
})

fig = px.line(
    df,
    x='Año',
    y=['Escenario A (sin sub)', 'Escenario B (con sub)'],
    markers=True,
    title='Comparación de ingresos: A vs B (3 años)'
)
fig.update_layout(template='plotly_white', font=dict(family='Segoe UI, Roboto, Arial', size=13))
fig.update_yaxes(title_text='Ingresos (USD)', tickformat='$,.0f')
fig.show()
df